In [1]:
// Golang implementation of a single hidden layer feed forward neural network
// based on the Python notebook for Make Your Own Neural Network by Tariq Rashid.

In [44]:
import (
    "encoding/csv"
    "fmt"
    "math"
    "os"
    "strconv"
    
    "gonum.org/v1/gonum/mat"
    "gonum.org/v1/gonum/stat/distuv"
)

In [60]:
// NewNeuralNetwork creates a `NeuralNetwork`.
func NewNeuralNetwork(inputNodes, hiddenNodes, outputNodes int, learningRate float64, activationFunc func(x float64) float64) *NeuralNetwork {
    return &NeuralNetwork{
        inputNodes:     inputNodes,
        hiddenNodes:    hiddenNodes,
        outputNodes:    outputNodes,
        learningRate:   learningRate,
        wih:            newRandomNormalMatrix(hiddenNodes, inputNodes, newNormal(inputNodes)),
        who:            newRandomNormalMatrix(outputNodes, hiddenNodes, newNormal(hiddenNodes)),
        activationFunc: activationFunc,
    }
}

// newNormal returns a normal (Gaussian) distribution with mean 0.0
// and standard deviation of the inverse square root of the number
// of incoming nodes n, in order to generate random real number
// initial weights in the range from -1/n^1/2 to 1/n^1/2.
func newNormal(incomingNodes int) distuv.Normal {
    return distuv.Normal{
        Mu: 0.0,
        Sigma: math.Pow(float64(incomingNodes), -0.5),
    }
}

// newRandomNormalMatrix returns a rows x cols matrix of weights
// initialized from the given normal distribution.
func newRandomNormalMatrix(rows, cols int, dist distuv.Normal) mat.Matrix {
    data := make([]float64, rows * cols)
    for i := 0; i < len(data); i++ {
        data[i] = dist.Rand()
    }
    
    return mat.NewDense(rows, cols, data)
}

// NeuralNetwork is a single hidden layer feed forward neural network with backpropogation.
type NeuralNetwork struct {
    inputNodes     int
    hiddenNodes    int
    outputNodes    int
    learningRate   float64
    wih            mat.Matrix  // Field `wih` is the link weight matrix from input nodes to hidden nodes.
    who            mat.Matrix  // Field `who` is the link weight matrix from hidden nodes to output nodes.
    activationFunc func(x float64) float64
}

func (n *NeuralNetwork) Train(inputs, outputs []float64) {
}

func (n *NeuralNetwork) Query(inputsList []float64) []float64 {
    return nil
}

In [29]:
const (
    inputNodes   = 784
    hiddenNodes  = 200
    outputNodes  = 10
    learningRate = 0.1
    epochs       = 5
)

activationFunc := func(x float64) float64 {
    // The activation function is the sigmoid function. That is,
    // f(x) = 1 / 1 + e^-x
    return 1.0 / (1.0 + math.Exp(-x))
}

n := NewNeuralNetwork(inputNodes, hiddenNodes, outputNodes, learningRate, activationFunc)

In [22]:
// Load the MNIST training data CSV file into a list.

func readAllCSV(path string) (records [][]string, err error) {
    file, err := os.Open("mnist_dataset/mnist_train_100.csv")
    if err != nil {
        return
    }
    
    defer file.Close()

    csvFile := csv.NewReader(file)
    records, err = csvFile.ReadAll()
    return
}

trainData, err := readAllCSV("mnist_dataset/mnist_train_100.csv")
if err != nil {
    panic(err)
}

label[0]: 5


12 <nil>

In [63]:
// Train the neural network for n epochs.

func trainRecord(record []string) {
    // The first element is the actual label.
    data := record[1:]

    // Rescale the input color values from [0, 255] to [0.01, 1.0].
    inputs := make([]float64, len(data))
    for i, c := range data {
        d, _ := strconv.Atoi(c)
        inputs[i] = (float64(d) / 255.0 * 0.99) + 0.01
    }

    // Initialize the target output nodes with the smallest (non-zero) value from the range [0.1, 0.99].
    targets := make([]float64, outputNodes)
    for i := 0; i < len(targets); i++ {
        targets[i] = 0.1
    }

    // Set the actual target for this training record with the highest value from the range [0.1, 0.99].
    label, _ := strconv.Atoi(record[0])
    targets[label] = 0.99

    n.Train(inputs, targets)
}

// for i := 0; i < epochs; i++ {
for i := 0; i < 1; i++ {
    for _, record := range trainData {
        trainRecord(record)
    }
}

In [25]:
// Load the MNIST test data CSV file into a list.

testData, err := readAllCSV("mnist_dataset/mnist_test_10.csv")
if err != nil {
    panic(err)
}

label[0]: 5


12 <nil>

In [8]:
// Test the neural network using a scorecard.

In [9]:
// The performance score is the fraction of correct answers,
// which will be computed and printed.